## より動的な音声生成のためのText-to-Speechの制御

従来の[TTS API](https://platform.openai.com/docs/guides/text-to-speech)には、生成される音声の声質を制御する機能がありません。例えば、テキストの段落を音声に変換したい場合、音声生成に関する具体的な指示を与えることはできませんでした。

[音声チャット補完](https://platform.openai.com/docs/guides/audio/quickstart)を使用すると、音声を生成する前に具体的な指示を与えることができます。これにより、APIに異なる速度、トーン、アクセントで話すよう指示することが可能になります。適切な指示により、これらの音声はより動的で自然、かつ文脈に適したものにすることができます。

### 従来のTTS

従来のTTSでは音声を指定することはできますが、トーン、アクセント、その他の文脈的な音声パラメータを指定することはできません。

In [2]:
from openai import OpenAI
client = OpenAI()

tts_text = """
Once upon a time, Leo the lion cub woke up to the smell of pancakes and scrambled eggs.
His tummy rumbled with excitement as he raced to the kitchen. Mama Lion had made a breakfast feast!
Leo gobbled up his pancakes, sipped his orange juice, and munched on some juicy berries.
"""

speech_file_path = "./sounds/default_tts.mp3"
response = client.audio.speech.create(
    model="tts-1-hd",
    voice="alloy",
    input=tts_text,
)

response.write_to_file(speech_file_path)

### Chat Completions TTS

Chat Completionsを使用すると、音声を生成する前に具体的な指示を与えることができます。以下の例では、子供向けの学習環境でイギリス英語のアクセントを生成しています。これは、アシスタントの声が学習体験にとって重要な教育アプリケーションで特に有用です。

In [4]:
import base64

speech_file_path = "./sounds/chat_completions_tts.mp3"
completion = client.chat.completions.create(
    model="gpt-4o-audio-preview",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "mp3"},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that can generate audio from text. Speak in a British accent and enunciate like you're talking to a child.",
        },
        {
            "role": "user",
            "content": tts_text,
        }
    ],
)

mp3_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open(speech_file_path, "wb") as f:
    f.write(mp3_bytes)

speech_file_path = "./sounds/chat_completions_tts_fast.mp3"
completion = client.chat.completions.create(
    model="gpt-4o-audio-preview",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "mp3"},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that can generate audio from text. Speak in a British accent and speak really fast.",
        },
        {
            "role": "user",
            "content": tts_text,
        }
    ],
)

mp3_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open(speech_file_path, "wb") as f:
    f.write(mp3_bytes)

### Chat Completions 多言語TTS

異なる言語のアクセントでオーディオを生成することもできます。以下の例では、特定のスペイン語ウルグアイアクセントでオーディオを生成します。

In [12]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "You are an expert translator. Translate any text given into Spanish like you are from Uruguay.",
        },
        {
            "role": "user",
            "content": tts_text,
        }
    ],
)
translated_text = completion.choices[0].message.content
print(translated_text)

speech_file_path = "./sounds/chat_completions_tts_es_uy.mp3"
completion = client.chat.completions.create(
    model="gpt-4o-audio-preview",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "mp3"},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that can generate audio from text. Speak any text that you receive in a Uruguayan spanish accent and more slowly.",
        },
        {
            "role": "user",
            "content": translated_text,
        }
    ],
)

mp3_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open(speech_file_path, "wb") as f:
    f.write(mp3_bytes)

Había una vez un leoncito llamado Leo que se despertó con el aroma de panqueques y huevos revueltos. Su pancita gruñía de emoción mientras corría hacia la cocina. ¡Mamá León había preparado un festín de desayuno! Leo devoró sus panqueques, sorbió su jugo de naranja y mordisqueó algunas bayas jugosas.


## 結論

生成された音声の声質を制御する能力は、より豊かな音声体験のための多くの可能性を開きます。以下のような多くの使用例があります：

- **表現力の向上**: 制御可能なTTSは、トーン、ピッチ、速度、感情の調整を可能にし、音声が異なる気分（例：興奮、落ち着き、緊急性）を伝えることができます。
- **語学学習と教育**: 制御可能なTTSは、アクセント、抑揚、発音を模倣することができ、正確なイントネーションと強調が重要な語学学習者や教育アプリケーションに有益です。
- **文脈に応じた音声**: 制御可能なTTSは、専門文書には正式なトーン、社会的な交流には親しみやすい会話スタイルなど、コンテンツの文脈に合わせて音声を適応させます。これにより、バーチャルアシスタントやチャットボットでより自然な会話を作り出すことができます。